# A Basic Trend-Following System for BTC Futures (Single Pass)

In this notebook we show a very simple strategy based on the crossing of two moving averages. The strategy can be used as starting point for developing more complex approaches. We use a fast single-pass implementation which however can lead to unintentional looking forward.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

We work with complete time series and define allocation weights at once for all points in time. The computation is fast but, since the time series is processed in a single pass, forward looking can occur. 

> Note that the performace of this system is not so good as the Sharpe ratio is lower than 1. If you can write a system with a Sharpe ratio larer than 1, save your result and click on the **Submit** button in your **Development** area! Do not forget the final call to the **write** function as in this example.

In [1]:
import xarray as xr

import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta

# load data:
data = qndata.cryptofutures.load_data(min_date="2013-04-01")

# calc weights:
close = data.sel(field="close")
ma_slow = qnta.lwma(close, 50)
ma_fast = qnta.lwma(close, 10)
weights = xr.where(ma_fast > ma_slow, 1, -1)

# normalize if total allocations larger than 1 and fill potentially missing values:
weights = qnout.clean(weights, data)

# calc stats (you can comments these lines for submission):
stats = qnstats.calc_stat(data, weights.sel(time=slice("2014-01-01", None)))
display(stats.to_pandas().tail())

# check that weights are correct:
qnout.check(weights, data)

# write results, necessary for submission:
qnout.write(weights)


ffill if the current price is None...
Check missed dates...
Ok.
Normalization...
Done.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-02-23,28.499508,-0.118298,0.725689,-0.148325,-0.815527,0.825048,0.598728,1.0,1.0,0.097391,34.755102
2021-02-24,29.463588,0.033828,0.725579,-0.119514,-0.815527,0.834880,0.605771,1.0,1.0,0.097412,34.755102
2021-02-25,29.227004,-0.008030,0.725388,-0.126584,-0.815527,0.832020,0.603537,1.0,1.0,0.097370,34.755102
2021-02-26,27.422813,-0.061730,0.725589,-0.180501,-0.815527,0.811611,0.588895,1.0,1.0,0.097334,34.755102
2021-03-01,28.905167,0.054056,0.725639,-0.136202,-0.815527,0.827156,0.600216,1.0,1.0,0.097339,35.880000


Check missed dates...
Ok.
Check the sharpe ratio...
Period: 2014-01-01 - 2021-03-01
Sharpe Ratio = 0.8271558411465563


ERROR! The sharpe ratio is too low. 0.8271558411465563 < 1


Check correlation.



WARNING! This strategy correlates with other strategies.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 1.0
Current sharpe ratio(3y): 0.6353498535432425

Write output: /root/fractions.nc.gz


# Multi Pass analog

 We use a multi-pass implementation which forbids forward looking and can be used for testing your idea.

We define a wrapper function for loading the data, and implement our strategy inside a function which returns allocation weights for a given point in time. Then we call the built-in backtesting function and perform the simulation avoiding potential looking-forward issues. 

> Note that the performace of this system is not so good as the Sharpe ratio is lower than 1. If you can write a system with a **Sharpe ratio larger than 1**, save your result and click on the **Submit** button in your **Development** area!

```python
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata



def load_data(period):
    """Loads the BTC Futures data for the BTC Futures contest"""
    return qndata.cryptofutures.load_data(tail=period, dims=("time","field","asset"))



def strategy(data):
    """System will go long when the slow-moving moving average is smaller than the
    fast-moving one; otherwise it will go short; we use fast implementations
    of linear-weighted moving averages based on numba which can be found in
    qnt/ta/wma.py
    """
    close= data.sel(field="close")
    ma_slow= qnta.lwma(close, 50).isel(time=-1)
    ma_fast= qnta.lwma(close, 10).isel(time=-1)
    return xr.where(ma_fast > ma_slow, 1, -1)



weights = qnbt.backtest(
    # BTC Futures contest:
    competition_type= "cryptofutures",
    load_data= load_data,
    # lookback in calendar days, it should be larger than the max. lookback used for
    # indicators, in this case 200 trading days:
    lookback_period= 365,
    start_date= "2014-01-01",
    strategy= strategy,
    analyze=True,
    build_plots=True
)
```